In [1]:
from typing import List, Tuple
from datetime import datetime, timedelta
import os
import cv2
import pickle
import itertools
import json
import multiprocessing
from math import nan

import altair as alt
import numpy.typing as npt
import numpy as np
import pandas as pd
from tqdm import tqdm

from utils.compose_factories.compose import compose_factory
from utils.data_dirs import data_dirs
from utils.get_videos import get_videos
from utils.frame_json_encoder import FrameJSONEncoder
from utils.typings import Frame
from utils.load_images import load_images
from utils.get_compose_and_videos import get_compose_and_videos

In [2]:
raw_dir, processed_dir, folder, EXPERIMENT_DATA, suffix, *_ = data_dirs(False)
print(raw_dir)
print(processed_dir)
print(folder)
print(EXPERIMENT_DATA)
print(suffix)

/home/chanwutk/data/raw/full-dataset/trainval
/home/chanwutk/data/processed/full-dataset/trainval
v1.0-trainval
/home/chanwutk/data/raw/scenic/experiment_data



In [3]:
interpolate = False

In [4]:
only_keyframe = True

In [5]:
fps = 12

In [6]:
compose, videos = get_compose_and_videos(fps, raw_dir, processed_dir, only_keyframe, interpolate)

100%|█████████████████████████████████████████████████████████████████████████████████████████| 1183790/1183790 [00:04<00:00, 280842.96it/s]


In [7]:
# compose = compose_factory(fps, raw_dir, processed_dir, interpolate)
# sample_data = pd.read_pickle(os.path.join(processed_dir, 'sample_data.pkl'))
# videos = get_videos(sample_data, only_keyframe=only_keyframe)

In [10]:
output = {}
misalignments = []
for i, name in enumerate(videos):
    print(f'{i} / {len(videos)}:', name)
    video = videos[name]

    filenames = []
    for f in video:
        filenames.append(f.filename)
    img_cache = load_images(raw_dir, filenames)

    assert len(set(f.location for f in video)) == 1, set(f.location for f in video)
    frames, filename, _misalignments = compose(video[0].location, name, video, img_cache)
    o = {
        "filename": filename,
        "start": video[0].timestamp,
        "location": video[0].location,
        "columns": [
            "cameraId",
            "frameId",  # sample data token
            "frameNum",
            "filename",
            "cameraTranslation",
            "cameraRotation",
            "cameraIntrinsic",
            "egoTranslation",
            "egoRotation",
            "timestamp",
            "cameraHeading",
            "egoHeading",
            "location",
        ],
        "frames": [
            tuple(video[f])
            for img, f in frames
        ]
    }
    output[name] = o
    print("done", filename)
base = os.path.join(processed_dir, 'videos')
frames_name = 'frames'
if only_keyframe:
    frames_name = 'keyframes-' + frames_name
with open(os.path.join(base, frames_name + '.pickle'), "wb") as f:
    pickle.dump(output, f)

with open(os.path.join(base, frames_name + '.pkl'), "wb") as f:
    pickle.dump(output, f)

with open(os.path.join(base, frames_name + '.json'), "w") as f:
    json.dump(output, f, indent=2, sort_keys=True, cls=FrameJSONEncoder)

0 / 5100: scene-0001-CAM_FRONT
Reading images


100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:00<00:00, 323260.42it/s]


Writing scene (/home/chanwutk/data/processed/full-dataset/trainval/videos/keyframe-singapore-onenorth-scene-0001-CAM_FRONT.mp4):


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:00<00:00, 40.22it/s]

done keyframe-singapore-onenorth-scene-0001-CAM_FRONT.mp4
1 / 5100: scene-0001-CAM_FRONT_RIGHT
Reading images


...

done keyframe-singapore-onenorth-scene-0004-CAM_BACK_RIGHT.mp4
21 / 5100: scene-0004-CAM_BACK
Reading images



100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:00<00:00, 208154.04it/s]


Writing scene (/home/chanwutk/data/processed/full-dataset/trainval/videos/keyframe-singapore-onenorth-scene-0004-CAM_BACK.mp4):


 98%|████████████████████████████████████████████████████████████████████████████████████████████████████▍  | 39/40 [00:00<00:00, 50.76it/s]


KeyboardInterrupt: 